In [2]:
## Imports

# General
import os
import os.path as path
import json
from glob import glob
import cv2 as cv2
import numpy as np
import pandas as pd
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
from tqdm import tqdm

%matplotlib inline

In [24]:
# Set data to be processed
DROOT = './0_data'

# Sources
RND_SRC = path.join(DROOT, "2_procfootage/360-cropped.jpg")

# Destination video
VID_DST = path.join(DROOT, "4_simsiam/video_sample.mp4")
IMG_DST = path.join(DROOT, "4_simsiam/video_sample.jpg")

In [4]:
base = plt.imread(RND_SRC)
height, width, layers = base.shape
print(base.shape)
r = 150

(1810, 6516, 3)


In [14]:
length = 100

def get_random_movement(size, main_dev, n_part, part_dev, start, bias, vmin, vmax):
    main = np.random.normal(0, main_dev, length)
    main_summed = np.array([ sum(main[0:i]) + start + bias for i in range(size)])
    parts = [np.minimum(vmax, np.maximum(vmin, np.random.normal(main_summed, part_dev)))
             for x in range(n_part)]
    
    return parts
    
combine_lists_of_lists = lambda x, y: [list(zip(x[i], y[i])) for i in range(len(x))]

part_1_x = get_random_movement(length, 200, 3, 150, -600, width/2, 0, width)
part_1_y = get_random_movement(length, 20, 3, 20, -100, height/2, 0, height)
part_1 = combine_lists_of_lists(part_1_x, part_1_y)

part_2_x = get_random_movement(length, 100, 4, 250, 400, width/2, 0, width)
part_2_y = get_random_movement(length, 20, 4, 20, 100, height/2, 0, height)
part_2 = combine_lists_of_lists(part_2_x, part_2_y)

part_3_x = get_random_movement(length, 250, 4, 200, 1000, width/2, 0, width)
part_3_y = get_random_movement(length, 20, 4, 20, -200, height/2, 0, height)
part_3 = combine_lists_of_lists(part_3_x, part_3_y)

part_4_x = get_random_movement(length, 100, 5, 300, -700, width/2, 0, width)
part_4_y = get_random_movement(length, 20, 5, 20, 400, height/2, 0, height)
part_4 = combine_lists_of_lists(part_4_x, part_4_y)

parts = pd.DataFrame(part_1 + part_2 + part_3 + part_4).T

parts.head(1)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,"(2747.9268398162167, 791.5812791465198)","(2701.6215841445633, 776.3770068435981)","(2469.953086963723, 840.9244930469127)","(3699.372379316033, 1018.526580158108)","(3658.242233001392, 1017.7134561996713)","(3493.3801135677327, 961.4287287301823)","(3686.674484158418, 967.3413946719255)","(4205.122482874769, 696.8465441975054)","(4251.195509612474, 701.2218927926936)","(4517.818109768861, 697.1769380264216)","(4182.67553250097, 728.5141692615217)","(2458.238230271785, 1294.0538856377289)","(2531.0094711350407, 1319.7223970608945)","(2316.9734311699904, 1308.1530238511796)","(2694.9252295182123, 1333.6133400543195)","(2700.9805589392236, 1301.7043040588667)"


In [27]:
fourcc = cv2.VideoWriter_fourcc(*'FMP4')
video = cv2.VideoWriter(VID_DST, fourcc, 10, (1440, 720))

def get_cmap(n, name='hsv'):
    return plt.cm.get_cmap(name, n)
cmap = get_cmap(len([x[0] for x in parts.iloc[0]]))

for frame in tqdm(parts.itertuples()):
    
    index = frame[0]
    pars = frame[1:]
    
    fig, ax = plt.subplots(figsize=(20,10))
    ax.imshow(base)
    x = [x[0] for x in pars]
    y = [x[1] for x in pars]
    ax.scatter(x, y, c=range(len(x)), cmap=cmap, linewidths=15, alpha=0.7)
    ax.axis('off')
    fig.tight_layout(pad=0, h_pad=0)
    ax.margins(0, y=0)

    fig.canvas.draw()
    image_from_plot = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
    image_from_plot = image_from_plot.reshape(fig.canvas.get_width_height()[::-1] + (3,))
    
    if index == 20:
        fig.savefig(IMG_DST, bbox_inches='tight', pad_inches = 0)
    
    video.write(cv2.cvtColor(image_from_plot, cv2.COLOR_BGR2RGB))
    plt.close()
    
video.release()

100it [01:24,  1.18it/s]
